In [ ]:
!pip install pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.5 MB/s eta 0:00:00


In [ ]:
!pip install dicom2nifti

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 777.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 49.2 MB/s eta 0:00:00


In [ ]:
!pip install nilearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 63.6 MB/s eta 0:00:00


In [ ]:
import pydicom
import dicom2nifti
import nilearn
from nilearn import plotting
import pandas as pd
import os

In [112]:
class Paciente():
  def __init__(self):
    self.__nombre = None
    self.__IdPaciente = None

  def verNombre(self):
    return self.__nombre

  def asignarNombre(self,ruta):
    self.__nombre = ruta[0x0010,0x0010].value

  def verIdPaciente(self):
    return self.__IdPaciente

  def asignarIdPaciente(self,ruta):
    self.__IdPaciente = ruta[0x0010,0x0020].value

class Estudio():
  def __init__(self):
    self.__IdEstudio = None
    self.fecha = None
    self.modalidad = None
    self.descripcion = None
    self.dimensionImg = None

  def verIdEstudio(self):
    return self.__IdEstudio

  def asignarIdEstudio(self,ruta):
    self.__IdEstudio = ruta[0x0020,0x0010].value

  def visualizarInfoEst(self,archivo):
    self.fecha = archivo[0x0008,0x0020].value
    self.modalidad = archivo[0x0008,0x0060].value
    self.descripcion = archivo[0x0008,0x1030].value
    data = {'Consultas': ['Fecha','Modalidad','Descripcion'],
            'Datos': [self.fecha,self.modalidad,self.descripcion]}

    self.tabla = pd.DataFrame(data,columns = ['Consultas','Datos'])
    self.tabla = self.tabla.assign(Numero = range(1,4))
    self.tabla = self.tabla.set_index('Item')
    return self.tabla

  def convertir(self,dicom):
    self.dicom = os.path.join(dicom)
    self.carpetaNueva = os.mkdir('./nifti')
    self.conversion = dicom2nifti.convert_directory(self.dicom,self.carpetaNueva)
    return self.conversion

  def visualizarImagen(self,nifti,forma):
    self.nombre = Sistema.cargarArchivo()
    self.read = nilearn.image.load_img(nifti)
    self.imagen = plotting.plot_anat(self.read, display_mode = forma, title = self.nombre, colorbar = True, cmap = 'inferno')

class Sistema():
  def __init__(self):
    self.est = Estudio()

  def cargarCarpeta(self):
    self.local = os.chdir(r'/content/drive/MyDrive/archivos') # esta ruta debe actualizarse, puesto que los archivos
    self.lista = []                                           # se encuentan en otra ruta

    for archivo in os.listdir(os.getcwd()):
      if archivo.endswith(archivo):
        self.rutas = os.path.join(os.getcwd(), archivo)
        if self.rutas in self.lista:
          continue
        else:
          self.lista.append(self.rutas)
    self.lista.sort()
    return self.lista

  def cargarArchivo(self,lista,posicion):
    for archivo in os.listdir(lista[posicion]):
      if archivo.endswith(archivo):
        self.ruta = os.path.join(lista[posicion],archivo)
        self.leer = pydicom.dcmread(self.ruta)
        return self.leer

  def crearData(self,idPaciente):
    self.estudio = baseDatos.setdefault(idPaciente,[])

  def addEstudio(self,idPaciente,idEstudio):
    self.add = baseDatos[idPaciente].append(idEstudio)

  def validarIdPaciente(self,idPaciente):
    while True:
      if idPaciente in baseDatos:
        print('  Paciente registrado.\n')
        return idPaciente
      else:
        idPaciente = input('  Paciente no registrado. Intente de nuevo: ')

  def validarIdEstudio(self, idEstudio, idPaciente):
    while True:
      existencia = False
      for i, a in baseDatos.items():
        if idEstudio in a:
          existencia = True
          if i == idPaciente:
            idEstudio = input('\n  Este estudio ya le pertenece al actual paciente.\n  Ingrese otro estudio: ')
          else:
            add = input('\n  Este estudio le pertenece a otro paciente.\n  ¿Desea continuar con este?\n  1. Sí.\n  2. No.\n\n  Ingrese una opción: ')
            try:
              add = int(add)
              if add == 1:
                return idEstudio
              elif add == 2:
                idEstudio = input('  Ingrese otro estudio: ')
              else:
                print('  Opción no válida. Intente nuevamente.')
            except ValueError:
              print('  Valor incorrecto. Ingrese un número válido.')

      if not existencia:
              idEstudio = input('  Este estudio no existe. Intente de nuevo: ')

  def validarNumero(self,numero):
    while True:
      if numero.isdigit():
        return int(numero)
      try:
        return float(numero)
      except ValueError:
          numero = input('\n  Ingrese un número: ')

  def cargarConsulta(self,idPac):
    for i in baseDatos.keys():
      if i == idPac:
        lista = list(baseDatos.keys())
        self.posicion = lista.index(idPac)
    # se cargan las carpetas y archivos
    for i in range(len(Sistema.cargarCarpeta(self))):
      self.carpeta = Sistema.cargarCarpeta(self)
      self.archivo = Sistema.cargarArchivo(self,self.carpeta,self.posicion)
      return self.archivo

  def cargarVisualizar(self,idPac):
    for i in baseDatos.keys():
      if i == idPac:
        lista = list(baseDatos.keys())
        self.posicion = lista.index(idPac)
    # se cargan las carpetas y archivos
        self.carpeta = Sistema.cargarCarpeta(self)
    return os.path.join(self.carpeta[self.posicion])


In [115]:
baseDatos = {}

def main():
  pac = Paciente()
  est = Estudio()
  sis = Sistema()

  # se cargan las carpetas del local
  for i in range(len(sis.cargarCarpeta())):
    carpeta = sis.cargarCarpeta()
    archivo = sis.cargarArchivo(carpeta,i)

  # se asocia la información para la creación de una base de datos
    nombre = pac.asignarNombre(archivo)
    idPaciente = pac.asignarIdPaciente(archivo)
    idPaciente = pac.verIdPaciente()
    idEstudio = est.asignarIdEstudio(archivo)
    idEstudio = est.verIdEstudio()
    asociacion = sis.crearData(idPaciente)
    asociacion = sis.addEstudio(idPaciente,idEstudio)

  while True:

    print('\n¡Bienvenido al Sistema de Gestion Avanzada de Imagenes DICOM!\n')

    menu = input('  ¿Qué desea realizar?\n\n  1. Añadir un estudio a paciente.\n  2. Consultar información del estudio.\n  3. Visualizar imagen médica. \n  4. Salir del sistema.\n\n  Ingrese una opción: ')
    menu = sis.validarNumero(menu)

    if menu == 1: # opcion para añadir estudio
      idPac = input('\n  Ingrese el ID del paciente: ')
      idPac = sis.validarIdPaciente(idPac)
      idEst = input('  Ingrese el ID del estudio a añadir: ')
      idEst = sis.validarIdEstudio(idEst,idPac)
      add = sis.addEstudio(idPac,idEst)
      print('¡Ha añadido un nuevo archivo!')

    if menu == 2: # opcion para consultar estudio
      menu1 = input('\n  Consultas\n\n  1. Consultar información especifíca de un estudio.\n  2. Consultar cantidad de estudios por paciente.\n\n  Ingrese una opción: ')
      menu1 = sis.validarNumero(menu1)

      if menu1 == 1: # consultar info especifica
        idPac = input('\n  Ingrese el ID del paciente: ')
        idPac = sis.validarIdPaciente(idPac)
        consulta = sis.cargarConsulta(idPac)
        tabla = est.visualizarInfoEst(consulta)
        print(tabla)

      if menu1 == 2: # cantidad de estudios
        idPac = input('\n  Ingrese el ID del paciente: ')
        idPac = sis.validarIdPaciente(idPac)
        cantidad = len(list(baseDatos[idPac]))
        print(f'El paciente ingresado posee {cantidad} estudio(s)')

      else:
        print('\n  Valor no válido.')

    if menu == 3: # opcion para visualizar imagen
      idPac = input('\n  Ingrese el ID del paciente: ')
      idPac = sis.validarIdPaciente(idPac)
      visual = sis.cargarVisualizar(idPac)
      convertir = est.convertir(visual)
      # esto para evitar un error por cargar una carpeta sin un elemento DICOM para leer,
      # en caso de volver a cargar esta celda. Al haber finalizado la opción de 3.
      eliminarCarpeta = os.rmdir('./nifti')

    if menu == 4: # opcion para salir
      print('\n¡Gracias por usar el Sistema! Hasta pronto...')
      break

if __name__=='__main__':
  main()


¡Bienvenido al Sistema de Gestion Avanzada de Imagenes DICOM!

  ¿Qué desea realizar?

  1. Añadir un estudio a paciente.
  2. Consultar información del estudio.
  3. Visualizar imagen médica. 
  4. Salir del sistema.

  Ingrese una opción: 4

¡Gracias por usar el Sistema! Hasta pronto...
